<h2>Load and use the exported model from before

In [1]:
# !pip install tensorflow
# !pip install ktrain

import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

import warnings, os, re, ktrain
warnings.filterwarnings("ignore")

<h4>Main dataset to classify

In [2]:
# --------- Main dataframe without NaN
df = pd.read_csv(r'texts.csv').dropna()
# Only arabic characters (no punctuations too)
df['texts'] = df.texts.progress_map(lambda x : ' '.join(re.findall(r'[\u0600-\u06FF]+', x)))
# Drop empty values (FINAL DF)
df = df.replace(r'^\s*$', np.nan, regex=True).dropna()
# Keep only texts with length of 30 and more
df = df[df.texts.str.len() >= 30]

# Main categories
categories = ['Culture','Diverse','Economy','Politics','Sport']

df

100%|██████████| 51087/51087 [00:00<00:00, 81479.20it/s]


,texts
0,قال تعالى إنّهُم يَرَوْنَهُ بعيداًونَراهُ قريبا
1,الجيش الروسي يباد في مدينة إيربين الآن
2,لم يتبقى الا سنوات قليلة
7,لم نسمع منذ فترة عن هجمات داعش في افغانستان سي...
8,الولايات المتحدة تخطط لدعم طالبان ضد داعش مسرح...
...,...
70952,اهلا وسهلا بكم احبتي الكرام قروب مثل قروب دكتو...
70953,اهلين
70954,اريد اكثر عدد يدخل هنا
70955,اهلا فيج اختي


<h3>Main predictor model

In [7]:
pred = ktrain.load_predictor('3arabert_classifier')
pred

In [9]:
# ~ 30 minutes
df['predicted_class'] = df.texts.progress_map(lambda x : pred.predict(x))
df['category'] = df['predicted_class'].progress_map(lambda x : categories[x])

# # OR 
# df['category'] = df.texts.progress_map(lambda x : categories[pred.predict(x)])

 64%|██████▍   | 19199/30094 [14:37<10:54, 16.64it/s]

In [ ]:
df.to_csv('final_classified_texts.csv', index=False)